<a href="https://colab.research.google.com/github/Mik1810/AI-assignment/blob/main/finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ta -q # Technical Analysis library

  Preparing metadata (setup.py) ... done


In [8]:
!pip install catboost -q # Yahoo Finance library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.7 MB/s eta 0:00:00


In [9]:
# Importing Libraries

# Data Handling
import pandas as pd
import numpy as np

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.subplots as sp
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Financial Data Analysis
import yfinance as yf
import ta
import quantstats as qs

# Machine Learning
from sklearn.metrics import confusion_matrix, roc_auc_score, auc, roc_curve

# Classification ML algorithms
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

# Feature Selection
from sklearn.feature_selection import SelectKBest, f_classif

# Oversampling minority class for imbalanced datasets
from imblearn.over_sampling import SMOTE

# Feature Importance
from sklearn.inspection import permutation_importance

# Hiding warnings
import warnings
warnings.filterwarnings("ignore")

In [17]:
lef = yf.download('LE=F', end = '2023-05-29',
                    interval = '1wk') # 1-week timeframe

[*********************100%%**********************]  1 of 1 completed


In [20]:
candlestick = go.Candlestick(x = lef.index,
                            open = lef['Open'],
                            high = lef['High'],
                            low = lef['Low'],
                            close = lef['Adj Close'],
                            increasing = dict(line=dict(color = 'skyblue')),
                            decreasing = dict(line=dict(color = 'darkblue')))

# Defining layout
layout = go.Layout(title = {'text': '<b>Live Cattle Futures</b> <br><sup>(March,2002 - May, 2023)</sup>',
                           'x': 0.5,
                           'xanchor': 'center',
                           'yanchor': 'top',
                           'font': {
                               'size': 25
                           }},
                  yaxis = dict(title = 'Price (USD)'),
                  xaxis = dict(title = 'Date'),
                  template = 'ggplot2',
                  xaxis_rangeslider_visible = True,
                  yaxis_gridcolor = 'lightgray',
                  yaxis_tickfont = dict(color = 'black'),
                  xaxis_tickfont = dict(color = 'black'),
                  margin = dict(t = 80, l = 50, r = 50, b = 50),
                  height = 650, width = 1000,
                  plot_bgcolor = 'white')

fig = go.Figure(data = [candlestick], layout = layout)

fig.show(renderer="colab")